#### Importando as bibliotecas necessarias

In [ ]:
from google.oauth2 import service_account
from google.cloud import bigquery
import pandas as pd
import pandas_gbq
import csv

In [ ]:
#Lendo o arquivo de receita.
df_receita = pd.read_csv(r"D:\projetos_python\esfera_gcp\arquivos_tratar\gdvReceitasExcel.csv", encoding='ISO-8859-1', sep = ',')

In [ ]:
#Lendo o arquivo de despesa
df_despesas = pd.read_csv(r"D:\projetos_python\esfera_gcp\arquivos_tratar\gdvDespesasExcel.csv", encoding='ISO-8859-1', sep = ',')

#### Lendo os arquivos e colocando em dataframes

In [ ]:
#Lendo o arquivo de receita.
df_receita = pd.read_csv(r"D:\projetos_python\esfera_gcp\arquivos_tratar\gdvReceitasExcel.csv", encoding='ISO-8859-1', sep = ',')

In [ ]:
#Lendo o arquivo de despesa
df_despesas = pd.read_csv(r"D:\projetos_python\esfera_gcp\arquivos_tratar\gdvDespesasExcel.csv", encoding='ISO-8859-1', sep = ',')

#### Checando o cabeçalho e o final do arquivo

In [ ]:
df_receita.head(2)

In [ ]:
df_receita.tail(2)

In [ ]:
df_despesas.head(2)

In [ ]:
df_despesas.tail(2)

#### Renomeando colunas

In [ ]:
df_receita = df_receita.rename(columns={'Fonte de Recursos':"fonte_recursos",'Receita':"tipo_receita",'Arrecadado até 02/02/2024':"arrecadado"})

In [ ]:
#Renomeando todas as colunas do df.
df_despesas = df_despesas.rename(columns={'Fonte de Recursos':"fonte_recursos",'Despesa':"tipo_despesa",'Liquidado':"liquidado"})

In [ ]:
#### Checando as colunas renomeadas
df_receita.head(1)

In [ ]:
df_despesas.head(1)

#### Removendo colunas com dados nulos

In [ ]:
#Removendo coluna Unnamed no dataframe receita
df_receita = df_receita.drop('Unnamed: 3', axis=1)

In [ ]:
df_despesas= df_despesas.drop('Unnamed: 3', axis=1)

#### Removendo linha com os totais

In [ ]:
df_receita.tail(1)

In [ ]:
df_receita = df_receita.drop(df_despesas.index[284])

In [ ]:
df_despesas.tail(1)

In [ ]:
df_despesas = df_despesas.drop(df_despesas.index[2034])

#### Melhorando os campos do tipo string

In [ ]:
def normalize_text(text):
    text = text.lower()
    text = text.replace('á', 'a').replace('à', 'a').replace('â', 'a').replace('ã', 'a')
    text = text.replace('é', 'e').replace('è', 'e').replace('ê', 'e')
    text = text.replace('í', 'i').replace('ì', 'i').replace('î', 'i')
    text = text.replace('ó', 'o').replace('ò', 'o').replace('ô', 'o').replace('õ', 'o')
    text = text.replace('ú', 'u').replace('ù', 'u').replace('û', 'u')
    text = text.replace('ç', 'c')
    text = text.replace('ñ', 'n')
    text = text.replace(' ', '_')
    return text

In [ ]:
df_receita['fonte_recursos'] = df_receita['fonte_recursos'].str.strip().apply(normalize_text)
df_receita['tipo_receita'] = df_receita['tipo_receita'].str.strip().apply(normalize_text)

In [ ]:
df_despesas['fonte_recursos'] = df_despesas['fonte_recursos'].str.strip().apply(normalize_text)
df_despesas['tipo_despesa'] = df_despesas['tipo_despesa'].str.strip().apply(normalize_text)

#### Removendo os espaços em branco e tratando casas decimais

In [ ]:
#Removendo os espaços em branco, tratando o problema nas casas decimais e convertendo para o tipo float.
df_receita['arrecadado'] = df_receita['arrecadado'].str.strip().replace({r'\.': '', ',': '.'}, regex=True).astype(float)

In [ ]:
df_despesas['liquidado'] = df_despesas['liquidado'].str.strip().replace({r'\.': '', ',': '.'}, regex=True).astype(float)

#### Checando o valor total

In [ ]:
df_receita['arrecadado'].sum()

In [ ]:
df_despesas['liquidado'].sum()

### Agora vamos fazer o envio desse dataset para o bigquery como solução de banco de dados.

In [ ]:
SCOPES = [
    'https://www.googleapis.com/auth/cloud-platform'
]

credentials = service_account.Credentials.from_service_account_file(
    r"C:\Users\Iago_\Downloads\mystudentproject-292517-e8c3c9cd7791.json"
)

pandas_gbq.context.credentials = credentials
pandas_gbq.project = "mystudentproject-292517"

In [ ]:
# Crie um cliente BigQuery
client = bigquery.Client(credentials=credentials, project="mystudentproject-292517")

In [ ]:
# Defina o nome do dataset (conjunto de dados)
dataset_id = "mystudentproject-292517.teste_esfera"

In [ ]:
# Crie o objeto Dataset
dataset = bigquery.Dataset(dataset_id)

In [ ]:
# Crie o dataset no BigQuery
dataset = client.create_dataset(dataset, timeout=30)

#### Criando a tabela db_receita

In [ ]:
# Criando o objeto table 
table_id = f"{dataset_id}.db_receita"

In [ ]:
#Definindo o schema da tabela
schema = [
    bigquery.SchemaField("fonte_recursos", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("tipo_receita", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("arrecadado", "FLOAT", mode="NULLABLE"),
]

#### Criando a tabela db_despesa

In [ ]:
table_id = f"{dataset_id}.db_despesa"

In [ ]:
schema = [
    bigquery.SchemaField("fonte_recursos", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("tipo_despesa", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("liquidado", "FLOAT", mode="NULLABLE"),
]

#### Execute a criação da tabela uma de cada vez

In [ ]:
# Crie o objeto Table referenciando o dataset e o nome da tabela
table = bigquery.Table(table_id, schema=schema)

In [ ]:
# Crie a tabela no BigQuery
table = client.create_table(table)

#### Realizando o envio para o bq

In [ ]:
pandas_gbq.to_gbq(df_despesas, 'teste_esfera.db_despesa', project_id='mystudentproject-292517', if_exists='replace')

In [ ]:
pandas_gbq.to_gbq(df_receita, 'teste_esfera.db_receita', project_id='mystudentproject-292517', if_exists='replace')